#Permite chequear que los XMLs asociados a las imágenes no tengan errores y visualizar los box para comprobar que sean correctos:


1) Cargar librerías:

In [ ]:
#@title Cargar Librerías
from IPython.display import Image, display
from PIL import Image as ImPIL
import numpy as np
import os
import cv2
import os.path
import datetime

import tensorflow as tf
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import xml.etree.cElementTree as ET

print ("Librerías cargadas.")

2) Montar el Drive:

In [ ]:
#@title
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

3) Llevar a cabo la prueba:

In [ ]:
#@title Definir parámetros a utilizar 

#@markdown Parámetros de Archivos:
levantarSoloDA =  True #@param {type:"boolean"}
filtrar_prefijo_inicial_caso_generado = "" #@param {type:"string"}
drive_path = "/content/gdrive/MyDrive/IA/demoObjDet-Carteles/Carteles/DA" #@param {type:"string"}

#@markdown Parámetros sobre Controles:
revisarObjetosEncimados = True #@param {type:"boolean"}

#@markdown Parámetros sobre Información a Mostrar:
cantMuestraMostrar =  5#@param {type:"integer"}
mostrarImagenOriginal = True #@param {type:"boolean"}
extraerObjetos = False #@param {type:"boolean"}
mostrarObjetosEnImagen = True #@param {type:"boolean"}
calcularCantTipoObjetos =  True #@param {type:"boolean"}

print("Parámetros definidos.")

import colorsys
# arma lista de colores para los recuados
# (filtra colores más "visibles")
colors = []
for n, c in ImageColor.colormap.items():
  hls = colorsys.rgb_to_hls(*map(lambda _: _ / 255., ImageColor.getrgb(c))  )  
  if hls[0] > 0.3 and hls[1] > 0.3 and hls[2] > 0.3:
    colors.append(c)
#colors = list(ImageColor.colormap.values())
#colors = sorted(colors, key=lambda _: _[0])
colors = sorted(colors)

print("Colores definidos: ", len(colors))

In [ ]:
#@title Determinar XML a revisar
drive.mount('/content/gdrive', force_remount=True)

# define la carpeta donde están las imágenes para procesar
data_dir_path = drive_path 
image_dir_path = data_dir_path

# levanta los XML  para procesar
all_xml_array = [ fn for fn in os.listdir( data_dir_path ) if fn.endswith('.xml')  ]
if levantarSoloDA and len(all_xml_array)>0:
  all_xml_array = [ fn for fn in all_xml_array if fn.startswith('da')  ]
if filtrar_prefijo_inicial_caso_generado != "" and len(all_xml_array)>0:
  all_xml_array = [ fn for fn in all_xml_array if fn.find('_'+ filtrar_prefijo_inicial_caso_generado +'_')>0 ]

# desordena y toma algunos al azar
if len(all_xml_array)>cantMuestraMostrar:
  np.random.shuffle(all_xml_array)
  mostrar_xml_array, _ = np.split(np.array(all_xml_array), [cantMuestraMostrar])
else:
  mostrar_xml_array = np.array(all_xml_array)

print("> XML a probar: ", len(all_xml_array))
print("> XML a mostrar: ", len(mostrar_xml_array))

if len(mostrar_xml_array) > 0:
  fechasCreacion = [ datetime.datetime.fromtimestamp(os.path.getmtime(data_dir_path + '/' + ar)) for ar in mostrar_xml_array]
  print("> Fechas de creación de XML a mostrar entre ",   min( fechasCreacion ), " y ", max( fechasCreacion ))


In [ ]:
#@title Probar XML

# vuelve a ordenar la lista para mostrar bien
all_xml_array.sort()

# arma hash para contar tipo objetos
diccTipoObjCant = {}

# función para cálculo de Intersection over Union (IoU) 
def calc_IoU( gt_bbox, pred_bbox):
    '''
    This function takes the predicted bounding box and ground truth bounding box and 
    return the IoU ratio
    '''
    x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt = gt_bbox
    x_topleft_p, y_topleft_p, x_bottomright_p, y_bottomright_p = pred_bbox
    
    if (x_topleft_gt > x_bottomright_gt) or (y_topleft_gt> y_bottomright_gt):
        #raise AssertionError("Ground Truth Bounding Box is not correct")
        return -1
    if (x_topleft_p > x_bottomright_p) or (y_topleft_p> y_bottomright_p):
        #raise AssertionError("Predicted Bounding Box is not correct",x_topleft_p, x_bottomright_p,y_topleft_p,y_bottomright_gt)
        return -1
         
    #if the GT bbox and predcited BBox do not overlap then iou=0
    if(x_bottomright_gt< x_topleft_p):
        # If bottom right of x-coordinate  GT  bbox is less than or above the top left of x coordinate of  the predicted BBox
        
        return 0.0
    if(y_bottomright_gt< y_topleft_p):  # If bottom right of y-coordinate  GT  bbox is less than or above the top left of y coordinate of  the predicted BBox
        
        return 0.0
    if(x_topleft_gt> x_bottomright_p): # If bottom right of x-coordinate  GT  bbox is greater than or below the bottom right  of x coordinate of  the predcited BBox
        
        return 0.0
    if(y_topleft_gt> y_bottomright_p): # If bottom right of y-coordinate  GT  bbox is greater than or below the bottom right  of y coordinate of  the predcited BBox
        
        return 0.0
    
    
    GT_bbox_area = (x_bottomright_gt -  x_topleft_gt + 1) * (  y_bottomright_gt -y_topleft_gt + 1)
    Pred_bbox_area =(x_bottomright_p - x_topleft_p + 1 ) * ( y_bottomright_p -y_topleft_p + 1)
    
    x_top_left =np.max([x_topleft_gt, x_topleft_p])
    y_top_left = np.max([y_topleft_gt, y_topleft_p])
    x_bottom_right = np.min([x_bottomright_gt, x_bottomright_p])
    y_bottom_right = np.min([y_bottomright_gt, y_bottomright_p])
    
    intersection_area = (x_bottom_right- x_top_left + 1) * (y_bottom_right-y_top_left  + 1)
    
    union_area = (GT_bbox_area + Pred_bbox_area - intersection_area)
   
    return intersection_area/union_area


# función auxiliar para hacer el chequeo
def checkValoresXML(valMin, valMax, limiteMax, descVal, xml_file, class_name):
  res = 0
  
  if valMin < 0:
    print("**", xml_file, '-', class_name, 'error: ', descVal, 'min [', valMin, '] < 0')
    res = res + 1
  
  if valMax < 0:
    print("**", xml_file, '-', class_name, 'error: ', descVal, 'max [', valMax, '] < 0')
    res = res + 1

  if valMin > limiteMax:
    print("**", xml_file, '-', class_name, 'error: ', descVal, 'min [', valMin, '] > límiteMáximo [', limiteMax, '].')
    res = res + 1
  
  if valMax > limiteMax:
    print("**", xml_file, '-', class_name, 'error: ', descVal, 'max [', valMax, ']  > límiteMáximo [', limiteMax, '].')
    res = res + 1

  if valMin < 0:
    print("**", xml_file, '-', class_name, 'error: ', descVal, 'min [', valMin, '] < 0.')
    res = res + 1
  
  if valMax < 0:
    print("**", xml_file, '-', class_name, 'error: ', descVal, 'max [', valMax, '] < 0.')
    res = res + 1

  if valMin >= valMax:
      if valMin > valMax:
          print("**", xml_file, '-', class_name, 'error: ', descVal, 'min [', valMin, '] > ', descVal, 'max [', valMax, '].')
          res = res + 1
      else:
          print("**", xml_file, '-', class_name, 'error: ', descVal, 'min [', valMin, '] = ', descVal, 'max[', valMax, '].')
          res = res + 1
  
  return res

# variable auxiliar
xmlConError = []
cantErroresTotales = 0
cantProcesados = 0 
# procesa los XMLs
for xml_file in all_xml_array:
    # variables auxiliares para chequear duplicados
    auxListCheckDupPos = []
    auxListCheckDupClass = []
    cantErrores = 0
    
    cantProcesados = cantProcesados + 1
    print("\n\n", cantProcesados,"> ", xml_file)
    if not os.path.isfile(data_dir_path + '/' + xml_file):
      print("\tno se encuentra el archivo xml!")
      continue    

    # decide si muestra la imagen asociada a este XML
    muestraImagen = ((xml_file in mostrar_xml_array) and mostrarObjetosEnImagen)

    # carga la info del XML original
    et = ET.parse(data_dir_path + '/' + xml_file)
    element = et.getroot()
    element_objs = element.findall('object') 
    element_filename = element.find('filename').text
    xmlWidth = element.find('size').find('width').text 
    xmlHeight = element.find('size').find('height').text 
    ##if not('.png' in element_filename):
    ##    element_filename  = element_filename + '.png'
    img_filename = os.path.join(image_dir_path, element_filename)
    if not os.path.isfile(img_filename):
      print("\tno se encuentra la imagen ", img_filename, "!")
      continue 

    if muestraImagen or mostrarImagenOriginal:
      # carga la imagen
      imageCargada = ImPIL.open(img_filename)
      if mostrarImagenOriginal: 
          display( imageCargada )

    # imagen auxiliar para mostrar recuadros y obtener tamaño
    img = tf.io.read_file(img_filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = img.numpy()
    image_pil = Image.fromarray(np.uint8(img)).convert("RGB")    
    im_width, im_height = image_pil.size
    draw = ImageDraw.Draw(image_pil)

    # controla ancho de la imagen y XML
    if xmlWidth=="" or not xmlWidth.isnumeric():
      print("**", xml_file, "- [size->Width] ", xmlWidth, " inválido!") 
      cantErrores = cantErrores + 1
    else:
      if im_width != int(xmlWidth):
        print("**", xml_file, "- [size->Width] ", xmlWidth, " no coincide con ancho de la imagen ", im_width) 
        cantErrores = cantErrores + 1
    # controla alto de la imagen y XML
    if xmlHeight=="" or not xmlHeight.isnumeric():
        print("**", xml_file, "- [size->Height] ", xmlHeight, " inválido!") 
        cantErrores = cantErrores + 1
    else:
      if im_height != int(xmlHeight):
        print("**", xml_file, "- [size->Height] ", xmlHeight, " no coincide con ancho de la imagen ", im_height) 
        cantErrores = cantErrores + 1

    # procesa los elementos en el archivo XML original para generar el nuevo
    for element_obj in element_objs:

        # obtiene la información actual de la imagen
        class_name = element_obj.find('name').text 
        if calcularCantTipoObjetos:
          # cuenta tipo de objeto
          if class_name in diccTipoObjCant:
            diccTipoObjCant[class_name] = diccTipoObjCant[class_name] + 1
          else:
            diccTipoObjCant[class_name] = 0

        # obtiene info del box actual
        obj_bbox = element_obj.find('bndbox')
        x1 = int(round(float(obj_bbox.find('xmin').text)))
        y1 = int(round(float(obj_bbox.find('ymin').text)))
        x2 = int(round(float(obj_bbox.find('xmax').text)))
        y2 = int(round(float(obj_bbox.find('ymax').text)))

        # realiza el chequeo del XML
        cantErrores = checkValoresXML(x1, x2, im_width, 'X', xml_file, class_name) + cantErrores
        cantErrores = checkValoresXML(y1, y2, im_height, 'Y', xml_file, class_name) + cantErrores         
        
        # controla que no haya duplicados identicos
        auxVec = (x1, y1, x2, y2)        
        if auxVec in auxListCheckDupPos:
            i = auxListCheckDupPos.index( auxVec )
            print("**", xml_file, '- [', class_name, ',', auxVec, '] error registro duplicado con [', auxListCheckDupClass[i], ',', auxListCheckDupPos[i], '].')
            cantErrores = cantErrores + 1
        else:
            # controla que no haya alguno con coordenadas muy parecidas
            if revisarObjetosEncimados:
                i = 0
                while i < len(auxListCheckDupPos):
                  iou = calc_IoU(auxVec, auxListCheckDupPos[i])
                  if iou >= 0.90:
                    print("**", xml_file, '- [', class_name, ',', auxVec, '] error posiciones muy cercanas (IoU ', iou, ') con [', auxListCheckDupClass[i], ',', auxListCheckDupPos[i], '].')
                    cantErrores = cantErrores + 1                    
                    break
                  i = i + 1                  
            # lo agrega a la lista            
            auxListCheckDupPos.append( auxVec )
            auxListCheckDupClass.append( class_name )
        
        if muestraImagen and extraerObjetos:
            # extrae los objetos de la imagen
            print("  --- ", class_name)  
            imBox = imageCargada.crop((x1,y1,x2,y2))
            display( imBox )

        if muestraImagen:
            # genera los recuadros correspondientes
            color = colors[(hash(class_name)) % len(colors)]
            if color == "#ffffff":
              color = "#000000"
            draw.line([(x1, y1), (x1, y2), (x2, y2), (x2, y1), (x1, y1)], width=4, fill=color)
             
    if cantErrores > 0:
      xmlConError.append( xml_file )
    ##print("\n== Cantidad de Errores encontrados: ", cantErrores)
    cantErroresTotales = cantErroresTotales + cantErrores 
    
    if muestraImagen:
       # muestra la imagen con los recuadros correspondientes
       ##np.copyto(img, np.array(image_pil))
       print("\n")
       imMostrar = Image.fromarray(np.array(image_pil), 'RGB')
       display( imMostrar )

print("\n\n== Cantidad de XMLs chequeados: ", cantProcesados)
print("\n== Cantidad de Errores Totales encontrados: ", cantErroresTotales)
print("\n== XMLs con errores: ", xmlConError)



In [ ]:
#@title Muestra Cantidades por Tipo de Objeto

if calcularCantTipoObjetos:
  # calcula cantidad total
  cantTotal = 0
  for obj in diccTipoObjCant.keys():
    cantTotal = cantTotal +  diccTipoObjCant[obj] 
  if cantTotal == 0:
    print("= NO SE ENCUENTRAN OBJETOS!")
  else:  
    # muestra cuenta tipo de objeto
    print("\n\n== Cantidad de Tipos de Objetos encontrados: ")
    for obj in diccTipoObjCant.keys():
      porc = round(diccTipoObjCant[obj]*100/cantTotal, 2) 
      print("\t\t", obj, ": ", diccTipoObjCant[obj], "(", porc, "%)")
    print("\n\t\t== TOTAL: ", cantTotal)
  print("\n")

In [ ]:
#@title OPCIONAL: Eliminar casos con problemas
# nota: se debe copiar la lista de xml obtenida de la ejecución anterior
arBorrar =  []
for ar in arBorrar:
 print(ar)
 os.remove(  data_dir_path + '/' + ar)
 os.remove(  image_dir_path + '/' + ar.replace(".xml", ".png"))

print("Casos eliminados: ", len(arBorrar))